# SQL tutorial using a database of drinks and bars

Notebook by [Eric Elmoznino](https://github.com/EricElmoznino/lighthouse_sql_tutorial)

## Things we will be doing:
- Creating a PostgreSQL database using the `psql` shell tool
- Importing existing data to fill the database (the `drinks.sql` file in this repository)
- Using the `psycopg2` Python package to interact with the database (we could have alternatively used a dedicated SQL IDE like SQL Workbench)
- Quering the database to explore SQL concepts such as:
    - Filtering, ordering, limiting, etc.
    - Joining tables
    - Grouping records
    - Aggregate functions
    
## Requirements:
- Install PostgreSQL (https://www.postgresql.org/download/) and give the `postgres` user a password)
- Add the PostgreSQL bin directory path to your PATH environmental variable.
- Install the `psycopg2` Python package in your anaconda virtual environment of choice. **Note**: you will need to install other dependencies beforehand. Look up which ones for your operating system.
    - Using conda: `conda install -c anaconda psycopg2`
    - Using pip: `pip install psycopg2`

# Creating the database using `psql`

1. Start up your terminal (Anaconda Prompt for Windows).
2. Switch to the "postgres" user that was created automatically when installing PostgreSQL (ignore if on Windows). \
`sudo -u postgres -i`
3. Start the `psql` shell tool. \
`psql`
    - Windows users use: `psql -U postgres`
4. List the PostgreSQL databases that exist on your computer (press "q" when you want to exit the list view). \
`\l`
5. Create a new empty database called "drinks". \
`CREATE DATABASE drinks;`
4. Confirm that the "drinks" database was created (press "q" when you want to exit the list view). \
`\l`
5. We can now leave the `psql` tool. \
`exit`
5. And also log out of the "postgres" user. \
`exit`

# Create a Python connection to the database and load in some data from the `drinks.sql` file

In [3]:
import psycopg2 #allows python to interact with postgres databases
import pandas as pd

# Create a connection to the drinks database we just made (substitute the password you set for the "postgres" user)
con = psycopg2.connect(database='drinks', user='postgres', password='',
                       host='127.0.0.1', port='5432')

cur = con.cursor() #used to execut queries

cur.execute(open('drinks.sql', 'r').read())
con.commit()

In [4]:
# Using triple quotations will ensure that everything inside the string
# is read as a character (i.e. no need to use escape characters)
query = """
SELECT * 
FROM drinks
LIMIT 5;
"""

# You first execute the query, then get it's result.
# Note: if you try to chain .execute().fetchall() in the same statement,
# the database my not have time to execute the query in time, which will
# result in you getting an error.
cur.execute(query)
response = cur.fetchall()

# Let's look at the format of the response we get back
print(f'This is the raw response we get back:\n{response}\n')

# You can alternatively use pandas to get a nicely formatted DataFrame
pandas_response = pd.read_sql_query(query, con)
print('This is the nicely structured pandas response:\n')
pandas_response

This is the raw response we get back:
[('drink 1', 'cocktail'), ('drink 2', 'wine'), ('drink 3', 'rum'), ('drink 4', 'cocktail'), ('drink 5', 'cocktail')]

This is the nicely structured pandas response:



/Users/niclapenna/opt/anaconda3/envs/lhl_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,drink_id,type
0,drink 1,cocktail
1,drink 2,wine
2,drink 3,rum
3,drink 4,cocktail
4,drink 5,cocktail


In [7]:
# Before doing anything else, let's create a function out
# of the things we're doing above
def execute_query(query_string, return_pandas=True):
    if return_pandas:
        response = pd.read_sql_query(query_string, con)
    else:
        cur.execute(query_string)
        response = cur.fetchall()
    return response


# Let's try out our function to make sure it does
# the same as what we have above
execute_query(query)

/Users/niclapenna/opt/anaconda3/envs/lhl_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,drink_id,type
0,drink 1,cocktail
1,drink 2,wine
2,drink 3,rum
3,drink 4,cocktail
4,drink 5,cocktail


# Some SQL references before we get started
![](images/sql-cheat-sheet.png)

## Order of operations *VERY USEFUL!!*
![](images/order_of_operations.png)

## Next let's take a peek at our data.
<table style="border: 5px; width: 100%">
 <tr>
    <td><b style="font-size:30px">menu_items</b></td>
    <td><b style="font-size:30px">orders</b></td>
    <td><b style="font-size:30px">drinks</b></td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>bar</b>: string</td>
    <td style="font-size:20px"><b>bar</b>: string</td>
    <td style="font-size:20px"><b>type</b>: string</td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>price</b>: real</td>
    <td style="font-size:20px"><b>person</b>: string</td>
    <td style="font-size:20px"></td>
 </tr>
 <tr>
    <td style="font-size:20px"></td>
    <td style="font-size:20px"><b>date</b>: string</td>
    <td style="font-size:20px"></td>
 </tr>
 <tr>
    <td style="font-size:20px"></td>
    <td style="font-size:20px"><b>quantity</b>: integer</td>
    <td style="font-size:20px"></td>
 </tr>
</table>

In [6]:
execute_query("""

FROM menu_items

""")

/Users/niclapenna/opt/anaconda3/envs/lhl_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,bar,drink_id,price
0,bar 1,drink 40,60.820200
1,bar 1,drink 4,17.112810
2,bar 1,drink 6,313.767550
3,bar 1,drink 44,65.825480
4,bar 1,drink 47,61.203575
...,...,...,...
103,bar 20,drink 39,9.153013
104,bar 20,drink 50,6.543673
105,bar 20,drink 12,64.098755
106,bar 20,drink 33,8.007992


# Time to start writing some queries

## Problem 1
Get the bar name and average price of drinks at each bar.
<table style="border: 5px; width: 100%">
 <tr>
    <td><b style="font-size:30px">menu_items</b></td>
    <td><b style="font-size:30px">orders</b></td>
    <td><b style="font-size:30px">drinks</b></td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>bar</b>: string</td>
    <td style="font-size:20px"><b>bar</b>: string</td>
    <td style="font-size:20px"><b>type</b>: string</td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>price</b>: real</td>
    <td style="font-size:20px"><b>person</b>: string</td>
    <td style="font-size:20px"></td>
 </tr>
 <tr>
    <td style="font-size:20px"></td>
    <td style="font-size:20px"><b>date</b>: string</td>
    <td style="font-size:20px"></td>
 </tr>
 <tr>
    <td style="font-size:20px"></td>
    <td style="font-size:20px"><b>quantity</b>: integer</td>
    <td style="font-size:20px"></td>
 </tr>
</table>

In [10]:

execute_query("""
SELECT bar, AVG(price) AS average_price
FROM menu_items
GROUP BY bar;""")

/Users/niclapenna/opt/anaconda3/envs/lhl_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,bar,average_price
0,bar 16,121.364365
1,bar 12,152.071032
2,bar 7,61.567487
3,bar 11,31.562558
4,bar 3,129.809513
5,bar 18,105.214779
6,bar 19,18.224434
7,bar 17,107.329703
8,bar 13,150.560439
9,bar 8,67.919044


## Problem 2
Get the bars with the top 5 average prices.
<table style="border: 5px; width: 100%">
 <tr>
    <td><b style="font-size:30px">menu_items</b></td>
    <td><b style="font-size:30px">orders</b></td>
    <td><b style="font-size:30px">drinks</b></td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>bar</b>: string</td>
    <td style="font-size:20px"><b>bar</b>: string</td>
    <td style="font-size:20px"><b>type</b>: string</td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>price</b>: real</td>
    <td style="font-size:20px"><b>person</b>: string</td>
    <td style="font-size:20px"></td>
 </tr>
 <tr>
    <td style="font-size:20px"></td>
    <td style="font-size:20px"><b>date</b>: string</td>
    <td style="font-size:20px"></td>
 </tr>
 <tr>
    <td style="font-size:20px"></td>
    <td style="font-size:20px"><b>quantity</b>: integer</td>
    <td style="font-size:20px"></td>
 </tr>
</table>

In [12]:
execute_query("""
SELECT bar, AVG(price) AS average_price
FROM menu_items
GROUP BY bar
ORDER BY average_price DESC
LIMIT 5;
""")

/Users/niclapenna/opt/anaconda3/envs/lhl_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,bar,average_price
0,bar 12,152.071032
1,bar 13,150.560439
2,bar 3,129.809513
3,bar 16,121.364365
4,bar 17,107.329703


## Problem 3
Get the bar with the cheapest drink, along with the drink type and price.
<table style="border: 5px; width: 100%">
 <tr>
    <td><b style="font-size:30px">menu_items</b></td>
    <td><b style="font-size:30px">orders</b></td>
    <td><b style="font-size:30px">drinks</b></td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>bar</b>: string</td>
    <td style="font-size:20px"><b>bar</b>: string</td>
    <td style="font-size:20px"><b>type</b>: string</td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>price</b>: real</td>
    <td style="font-size:20px"><b>person</b>: string</td>
    <td style="font-size:20px"></td>
 </tr>
 <tr>
    <td style="font-size:20px"></td>
    <td style="font-size:20px"><b>date</b>: string</td>
    <td style="font-size:20px"></td>
 </tr>
 <tr>
    <td style="font-size:20px"></td>
    <td style="font-size:20px"><b>quantity</b>: integer</td>
    <td style="font-size:20px"></td>
 </tr>
</table>

In [17]:
execute_query("""
SELECT price, type, bar
FROM menu_items
JOIN drinks ON menu_items.drink_id=drinks.drink_id
ORDER BY price
LIMIT 1
""")

,price,type,bar
0,3.477886,soda,bar 18


## Problem 4
Get the number of beers sold by each bar in descending order (number of beers sold, not the number of beers on their menu).
<table style="border: 5px; width: 100%">
 <tr>
    <td><b style="font-size:30px">menu_items</b></td>
    <td><b style="font-size:30px">orders</b></td>
    <td><b style="font-size:30px">drinks</b></td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>bar</b>: string</td>
    <td style="font-size:20px"><b>bar</b>: string</td>
    <td style="font-size:20px"><b>type</b>: string</td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>price</b>: real</td>
    <td style="font-size:20px"><b>person</b>: string</td>
    <td style="font-size:20px"></td>
 </tr>
 <tr>
    <td style="font-size:20px"></td>
    <td style="font-size:20px"><b>date</b>: string</td>
    <td style="font-size:20px"></td>
 </tr>
 <tr>
    <td style="font-size:20px"></td>
    <td style="font-size:20px"><b>quantity</b>: integer</td>
    <td style="font-size:20px"></td>
 </tr>
</table>

In [19]:
execute_query("""
SELECT SUM(quantity), bar
FROM orders
JOIN drinks ON orders.drink_id=drinks.drink_id
WHERE type LIKE '%beer%'
GROUP BY bar
""")

/Users/niclapenna/opt/anaconda3/envs/lhl_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,sum,bar
0,66,bar 10
1,80,bar 11
2,36,bar 17
3,79,bar 2
4,176,bar 20
5,108,bar 3
6,111,bar 5


## Problem 5
For each person, find the bars they visit, and the type(s) and price(s) of the drink(s) they drink during those visits.
<table style="border: 5px; width: 100%">
 <tr>
    <td><b style="font-size:30px">menu_items</b></td>
    <td><b style="font-size:30px">orders</b></td>
    <td><b style="font-size:30px">drinks</b></td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
    <td style="font-size:20px"><b>drink_id</b>: string</td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>bar</b>: string</td>
    <td style="font-size:20px"><b>bar</b>: string</td>
    <td style="font-size:20px"><b>type</b>: string</td>
 </tr>
 <tr>
    <td style="font-size:20px"><b>price</b>: real</td>
    <td style="font-size:20px"><b>person</b>: string</td>
    <td style="font-size:20px"></td>
 </tr>
 <tr>
    <td style="font-size:20px"></td>
    <td style="font-size:20px"><b>date</b>: string</td>
    <td style="font-size:20px"></td>
 </tr>
 <tr>
    <td style="font-size:20px"></td>
    <td style="font-size:20px"><b>quantity</b>: integer</td>
    <td style="font-size:20px"></td>
 </tr>
</table>

In [21]:
execute_query("""
SELECT o.person, o.bar, d.type, d.drink_id, m.price
FROM orders AS o
JOIN menu_items AS m ON (o.drink_id = m.drink_id AND o.bar = m.bar) 
JOIN drinks AS d ON o.drink_id = d.drink_id
GROUP BY o.person, o.bar, d.type, d.drink_id, m.price;
 """)

/Users/niclapenna/opt/anaconda3/envs/lhl_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,person,bar,type,drink_id,price
0,person 49,bar 20,beer,drink 50,6.543673
1,person 38,bar 19,whisky,drink 42,34.507183
2,person 45,bar 17,wine,drink 2,302.857150
3,person 56,bar 19,whisky,drink 42,34.507183
4,person 43,bar 20,beer,drink 50,6.543673
...,...,...,...,...,...
1363,person 44,bar 9,whisky,drink 34,74.957020
1364,person 42,bar 4,vodka,drink 41,34.621815
1365,person 36,bar 12,wine,drink 37,443.173860
1366,person 40,bar 7,whisky,drink 28,91.364480


# Below are my answers

## Answer 1

In [ ]:
execute_query("""
SELECT bar, AVG(price) AS avg_price
FROM menu_items
GROUP BY bar;
""")

# when you use an aggregate function, should use a GROUP BY (or it will aggregate the entire column)
# whenever you use a GROUP BY, your selected columns need an aggregation (unless it's the GROUP BY column)
# (SELECT drink_id gives error)

## Answer 2

In [ ]:
execute_query("""
SELECT bar, AVG(price) AS avg_price
FROM menu_items
GROUP BY bar
ORDER BY avg_price DESC
LIMIT 5;
""")

# Can also use 'ORDER BY 2 DESC'

## Answer 3

In [ ]:
execute_query("""
SELECT menu_items.bar, drinks.type, menu_items.price
FROM menu_items JOIN drinks ON menu_items.drink_id=drinks.drink_id
ORDER BY price ASC
LIMIT 1;
""")

## Answer 4

In [ ]:
execute_query("""
SELECT orders.bar, SUM(orders.quantity) as beers_sold 
FROM orders JOIN drinks ON drinks.drink_id = orders.drink_id
WHERE drinks.type = 'beer'
GROUP BY orders.bar
ORDER BY beers_sold DESC;
""")

# Can also use: WHERE drinks.type LIKE '%beer%'

## Answer 5

In [ ]:
execute_query("""
SELECT o.person, o.bar, d.type, d.drink_id, m.price
FROM orders AS o
JOIN menu_items AS m ON (o.drink_id = m.drink_id AND o.bar = m.bar) 
JOIN drinks AS d ON o.drink_id = d.drink_id
GROUP BY o.person, o.bar, d.type, d.drink_id, m.price;
""")

# Remember to close your database connection when you're done.
**Note**: If you run this cell then try to run queries, it won't work.
You will have to re-run the cell that initialized the connection.

In [22]:
con.close()